# Fitting a basic Cox time-varying model to the NBA data

In [1]:
import sys
import logging
from pathlib import Path

import pandas as pd
from prefect import Flow

from nba_survival.model.tasks import (
    SurvivalData,
    SegmentData,
    InitializeLifelines,
    FitLifelinesModel,
    CollapseData,
    PredictLifelines,
    ConcordanceIndex,
    HyperparameterTuning
)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

## Load the data

In [2]:
df = pd.concat(
    pd.read_csv(fpath, sep="|", dtype={"GAME_ID": str}, index_col=0)
    for fpath in Path("..", "nba-data", "2018-19", "model-data").glob("data_*.csv")
).reset_index(drop=True)

In [3]:
df.head(n=5)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,HOME_W_PCT,VISITOR_W_PCT,HOME_GAMES_IN_LAST_3_DAYS,VISITOR_GAMES_IN_LAST_3_DAYS,HOME_GAMES_IN_LAST_5_DAYS,VISITOR_GAMES_IN_LAST_5_DAYS,HOME_GAMES_IN_LAST_7_DAYS,VISITOR_GAMES_IN_LAST_7_DAYS,HOME_LINEUP_PLUS_MINUS,VISITOR_LINEUP_PLUS_MINUS
0,0021800001,4,10,0,1,8:03 PM,12:00,Jump Ball Horford vs. Embiid: Tip to Simmons,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,-0.1
1,0021800001,8,4,0,1,8:04 PM,11:40,CELTICS Rebound,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,-0.1
2,0021800001,10,2,1,1,8:04 PM,11:15,MISS Tatum 25' 3PT Jump Shot,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,-0.1
3,0021800001,11,4,0,1,8:04 PM,11:13,NaN,NaN,Saric REBOUND (Off:0 Def:1),...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,-0.1
4,0021800001,12,5,1,1,8:04 PM,11:08,Hayward STEAL (1 STL),NaN,Simmons Bad Pass Turnover (P1.T1),...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,-0.1


## Build a basic model (default values)

In [4]:
format_data = SurvivalData(name="Convert input data to range form")
segdata = SegmentData(name="Create train and test data")
model = InitializeLifelines(name="Initialize Cox model")
trained = FitLifelinesModel(name="Fit Cox model")
collapsed = CollapseData(name="Collapse training data")
predict = PredictLifelines(name="Predict partial hazard")
concord = ConcordanceIndex(name="Calculate C-Index")

In [5]:
with Flow(name="My model pipeline") as flow:
    alldata = format_data(df)
    data = segdata(alldata)
    model_obj = model()
    fitted = trained(model=model_obj, data=data["train"])
    testing = collapsed(data["test"])
    predt = predict(model=fitted, data=testing)
    cind = concord(data=testing, predt=predt)

In [6]:
output = flow.run()

[2021-02-12 09:36:56+0000] INFO - prefect.FlowRunner | Beginning Flow run for 'My model pipeline'
INFO:prefect.FlowRunner:Beginning Flow run for 'My model pipeline'
[2021-02-12 09:36:56+0000] INFO - prefect.TaskRunner | Task 'Initialize Cox model': Starting task run...
INFO:prefect.TaskRunner:Task 'Initialize Cox model': Starting task run...
[2021-02-12 09:36:56+0000] INFO - prefect.TaskRunner | Task 'Initialize Cox model': Finished task run for task with final state: 'Success'
INFO:prefect.TaskRunner:Task 'Initialize Cox model': Finished task run for task with final state: 'Success'
[2021-02-12 09:36:56+0000] INFO - prefect.TaskRunner | Task 'Convert input data to range form': Starting task run...
INFO:prefect.TaskRunner:Task 'Convert input data to range form': Starting task run...


/usr/local/lib/python3.8/site-packages/lifelines/utils/__init__.py:1452: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cv[duration_col] += delay


[2021-02-12 09:37:21+0000] INFO - prefect.TaskRunner | Task 'Convert input data to range form': Finished task run for task with final state: 'Success'
INFO:prefect.TaskRunner:Task 'Convert input data to range form': Finished task run for task with final state: 'Success'
[2021-02-12 09:37:21+0000] INFO - prefect.TaskRunner | Task 'Create train and test data': Starting task run...
INFO:prefect.TaskRunner:Task 'Create train and test data': Starting task run...
[2021-02-12 09:37:21+0000] INFO - prefect.Create train and test data | Setting the seed to 42
INFO:prefect.Create train and test data:Setting the seed to 42
[2021-02-12 09:37:22+0000] INFO - prefect.Create train and test data | Dataset ``train`` has 1044 rows
INFO:prefect.Create train and test data:Dataset ``train`` has 1044 rows
[2021-02-12 09:37:22+0000] INFO - prefect.Create train and test data | Dataset ``test`` has 185 rows
INFO:prefect.Create train and test data:Dataset ``test`` has 185 rows
[2021-02-12 09:37:22+0000] INFO - p

/usr/local/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()


Iteration 5: norm_delta = 0.00000, step_size = 1.00000, ll = -3605.24794, newton_decrement = 0.00000, seconds_since_start = 1.0Convergence completed after 5 iterations.
[2021-02-12 09:37:23+0000] INFO - prefect.TaskRunner | Task 'Fit Cox model': Finished task run for task with final state: 'Success'
INFO:prefect.TaskRunner:Task 'Fit Cox model': Finished task run for task with final state: 'Success'
[2021-02-12 09:37:23+0000] INFO - prefect.TaskRunner | Task 'Predict partial hazard': Starting task run...
INFO:prefect.TaskRunner:Task 'Predict partial hazard': Starting task run...
[2021-02-12 09:37:23+0000] INFO - prefect.TaskRunner | Task 'Predict partial hazard': Finished task run for task with final state: 'Success'
INFO:prefect.TaskRunner:Task 'Predict partial hazard': Finished task run for task with final state: 'Success'
[2021-02-12 09:37:23+0000] INFO - prefect.TaskRunner | Task 'Calculate C-Index': Starting task run...
INFO:prefect.TaskRunner:Task 'Calculate C-Index': Starting tas

## Create a model with hyperparameter tuning

In [7]:
format_data = SurvivalData(name="Convert input data to range form")
segdata = SegmentData(name="Create train and test data")
tuning = HyperparameterTuning(name="Hyperparameter tuning")
model = InitializeLifelines(name="Initialize Cox model")
trained = FitLifelinesModel(name="Fit Cox model")
collapsed_tune = CollapseData(name="Collapse tune data")
collapsed_train = CollapseData(name="Collapse test data")
predict = PredictLifelines(name="Predict partial hazard")
concord = ConcordanceIndex(name="Calculate C-Index")

In [8]:
with Flow(name="My model pipeline") as flow:
    alldata = format_data(df)
    data = segdata(alldata, splits=[0.8, 0.9], keys=["train", "tune", "test"])
    tune = collapsed_tune(data["tune"])
    params = tuning(data["train"], tune)
    model_obj = model(params["best"])
    fitted = trained(model=model_obj, data=data["train"])
    testing = collapsed(data["test"])
    predt = predict(model=fitted, data=testing)
    cind = concord(data=testing, predt=predt)

In [9]:
output = flow.run()

[2021-02-12 09:37:24+0000] INFO - prefect.FlowRunner | Beginning Flow run for 'My model pipeline'
INFO:prefect.FlowRunner:Beginning Flow run for 'My model pipeline'
[2021-02-12 09:37:24+0000] INFO - prefect.TaskRunner | Task 'Convert input data to range form': Starting task run...
INFO:prefect.TaskRunner:Task 'Convert input data to range form': Starting task run...


/usr/local/lib/python3.8/site-packages/lifelines/utils/__init__.py:1452: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cv[duration_col] += delay


[2021-02-12 09:37:49+0000] INFO - prefect.TaskRunner | Task 'Convert input data to range form': Finished task run for task with final state: 'Success'
INFO:prefect.TaskRunner:Task 'Convert input data to range form': Finished task run for task with final state: 'Success'
[2021-02-12 09:37:49+0000] INFO - prefect.TaskRunner | Task 'Create train and test data': Starting task run...
INFO:prefect.TaskRunner:Task 'Create train and test data': Starting task run...
[2021-02-12 09:37:49+0000] INFO - prefect.Create train and test data | Setting the seed to 42
INFO:prefect.Create train and test data:Setting the seed to 42
[2021-02-12 09:37:49+0000] INFO - prefect.Create train and test data | Dataset ``train`` has 983 rows
INFO:prefect.Create train and test data:Dataset ``train`` has 983 rows
[2021-02-12 09:37:49+0000] INFO - prefect.Create train and test data | Dataset ``tune`` has 123 rows
INFO:prefect.Create train and test data:Dataset ``tune`` has 123 rows
[2021-02-12 09:37:49+0000] INFO - pre

/usr/local/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()



Iteration 1: norm_delta = 0.02785, step_size = 0.90000, ll = -304491.73094, newton_decrement = 12.24364, seconds_since_start = 0.2
Iteration 2: norm_delta = 0.00429, step_size = 0.90000, ll = -204214.51213, newton_decrement = 0.39001, seconds_since_start = 0.4
Iteration 3: norm_delta = 0.00616, step_size = 0.90000, ll = -137372.10085, newton_decrement = 1.10979, seconds_since_start = 0.6
Iteration 4: norm_delta = 0.00492, step_size = 0.88200, ll = -92810.19967, newton_decrement = 1.04053, seconds_since_start = 0.8
Iteration 5: norm_delta = 0.00369, step_size = 0.86436, ll = -63102.28269, newton_decrement = 0.87515, seconds_since_start = 1.0
Iteration 6: norm_delta = 0.00316, step_size = 1.00000, ll = -43297.00769, newton_decrement = 0.69892, seconds_since_start = 1.2
Iteration 7: norm_delta = 0.00180, step_size = 1.00000, ll = -30093.27866, newton_decrement = 0.33540, seconds_since_start = 1.4
Iteration 8: norm_delta = 0.00124, step_size = 1.00000, ll = -21290.95108, newton_decrement =

...

Iteration 1: norm_delta = 0.20155, step_size = 0.90000, ll = -7803.97554, newton_decrement = 88.10259, seconds_since_start = 0.4
Iteration 2: norm_delta = 0.01330, step_size = 0.90000, ll = -6346.69948, newton_decrement = 0.39868, seconds_since_start = 0.7
Iteration 3: norm_delta = 0.00735, step_size = 0.90000, ll = -5435.24737, newton_decrement = 0.12357, seconds_since_start = 1.0
Iteration 4: norm_delta = 0.01303, step_size = 1.00000, ll = -4830.46634, newton_decrement = 0.33165, seconds_since_start = 1.2
Iteration 5: norm_delta = 0.01411, step_size = 0.98000, ll = -4430.07379, newton_decrement = 0.42604, seconds_since_start = 1.6
Iteration 6: norm_delta = 0.01516, step_size = 0.96040, ll = -4166.15756, newton_decrement = 0.53424, seconds_since_start = 1.8
Iteration 7: norm_delta = 0.01454, step_size = 0.94119, ll = -3992.90603, newton_decrement = 0.54153, seconds_since_start = 2.1
Iteration 8: norm_delta = 0.01215, step_size = 0.92237, ll = -3879.52720, newton_decrement = 0.46142, s

/usr/local/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()



Iteration 1: norm_delta = 0.04356, step_size = 0.90000, ll = -22278.80106, newton_decrement = 19.15004, seconds_since_start = 0.2
Iteration 2: norm_delta = 0.00296, step_size = 0.90000, ll = -16062.85399, newton_decrement = 0.09162, seconds_since_start = 0.4
Iteration 3: norm_delta = 0.00163, step_size = 0.90000, ll = -11931.96987, newton_decrement = 0.02884, seconds_since_start = 0.6
Iteration 4: norm_delta = 0.00304, step_size = 1.00000, ll = -9178.71498, newton_decrement = 0.08694, seconds_since_start = 0.8
Iteration 5: norm_delta = 0.00357, step_size = 0.98000, ll = -7343.91553, newton_decrement = 0.13702, seconds_since_start = 1.0
Iteration 6: norm_delta = 0.00429, step_size = 0.96040, ll = -6121.55797, newton_decrement = 0.23112, seconds_since_start = 1.2
Iteration 7: norm_delta = 0.00482, step_size = 0.94119, ll = -5307.51355, newton_decrement = 0.35218, seconds_since_start = 1.5
Iteration 8: norm_delta = 0.00505, step_size = 0.92237, ll = -4765.60306, newton_decrement = 0.47902

/usr/local/lib/python3.8/site-packages/lifelines/utils/__init__.py:1099: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  problem_columns = (censors_only | deaths_only).difference(total).tolist()


Iteration 30: norm_delta = 0.00001, step_size = 1.00000, ll = -3457.88245, newton_decrement = 0.00000, seconds_since_start = 6.4Convergence completed after 30 iterations.
[2021-02-12 09:48:11+0000] INFO - prefect.TaskRunner | Task 'Fit Cox model': Finished task run for task with final state: 'Success'
INFO:prefect.TaskRunner:Task 'Fit Cox model': Finished task run for task with final state: 'Success'
[2021-02-12 09:48:11+0000] INFO - prefect.TaskRunner | Task 'Predict partial hazard': Starting task run...
INFO:prefect.TaskRunner:Task 'Predict partial hazard': Starting task run...
[2021-02-12 09:48:11+0000] INFO - prefect.TaskRunner | Task 'Predict partial hazard': Finished task run for task with final state: 'Success'
INFO:prefect.TaskRunner:Task 'Predict partial hazard': Finished task run for task with final state: 'Success'
[2021-02-12 09:48:11+0000] INFO - prefect.TaskRunner | Task 'Calculate C-Index': Starting task run...
INFO:prefect.TaskRunner:Task 'Calculate C-Index': Starting t